# Scraping for Question 1
Scraping necessary data to look at the impact of the opening kill on the win rate of a round and the different factors of an opening kill.

This data is taken from DreamTeam.gg, which is an analysis tool used for CS:GO matches. CS:GO matches can be saved afterwards in what are known as "Demo" files. DreamTeam uses these demo files to analyse individual matches and pull statistics that are not available on HLTV.org. The data scraped in this notebook is the indivdual round winner of each match.

## 2nd Scrape: Round Winner (DreamTeam)
The first step is to import the necessary libraries that will be used in this notebook.

In [1]:
# Import libraries
import pandas as pd
import urllib
import urllib.request
from time import time, sleep
import re
from loguru import logger
import json
from pprint import pprint

Next, functions are written that will be used later for various purposes.

In [2]:
# Functions

# Uses urllib to retrieve webpage and scrape JSON data
def get_json(url):
    # Retrieve HTML
    req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"})
    page = urllib.request.urlopen(req).read()

    # Sleep to prevent overflowing website
    sleep(1)

    # Output of page returned is JSON, no need for BeautifulSoup
    data = json.loads(page)
    return data

# Build the URL needed for scraping
def get_url(MATCHCODE):
    url = "https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/" + MATCHCODE + "/rounds"
    return url
    
# Scrape round winner details
def scrape_round_winner(matchID, event, MATCHCODE):
    # Dataframe that contains details for each round
    df = pd.DataFrame(columns=["Event", "Match ID", "Round", "Winner", "Winner Side", "Match Winner"])
    
    # Get URL
    url = get_url(MATCHCODE)
        
    # Logger message
    logger.info('Scraping new match: %s' % url)
    
    # Get JSON data
    data = get_json(url)
    
    # Get round count for iterating through first kills
    roundCount = data["total"]
    
    # Get details needed from each round and append to DataFrame
    for x in range(roundCount):
        roundNum = data["items"][x]["number"]
        roundWinner = data["items"][x]["winner_team"]
        winnerSide = data["items"][x]["winner_side"]
        matchWinner = data["items"][roundCount-1]["winner_team"]
        df = df.append({"Event": event, "Match ID": matchID, "Round": roundNum, 
                        "Winner": roundWinner, "Winner Side": winnerSide, 
                        "Match Winner": matchWinner}, ignore_index=True)
        
    return df

Unlike the previous scrape where the URLs to individual maps were freely available, I had to manually upload the demo files to DreamTeam.gg. After the demo is analysed, the statistics are stored on the website and a URL is given. I combined these URLs with the corresponding HLTV Match ID and event name in a CSV file. Unfortunately, CS:GO recieved an update in 2014 that changed the way demo files are saved. This means that the first two majors, **EMS ONE Katowice 2014 and DREAMHACK Jönköping 2013**, could not be analysed using DreamTeam.

To begin, this CSV file is read in and saved to a DataFrame.

In [3]:
# Read in DreamTeam links from CSV file
dfDTLinks = pd.read_csv("DREAM_TEAM_LINKS.csv")
dfDTLinks.head(5)

,Event,Dream Team URL,Match ID
0,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,82145.0
1,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5c89...,82143.0
2,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5cab...,82123.0
3,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5c81...,82121.0
4,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5c88...,82120.0


The DataFrame is checked for null values and those null values are then dropped. These correspond to the first two majors and several other matches where the demo file was corrupt.

In [4]:
print("-----------------------------------------")
print("DataFrame Info")
print(dfDTLinks.info())
print("-----------------------------------------\nDataFrame Null Values")
print(dfDTLinks.isnull().sum())
print("-----------------------------------------")

-----------------------------------------
DataFrame Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621 entries, 0 to 620
Data columns (total 3 columns):
Event             621 non-null object
Dream Team URL    542 non-null object
Match ID          545 non-null float64
dtypes: float64(1), object(2)
memory usage: 14.6+ KB
None
-----------------------------------------
DataFrame Null Values
Event              0
Dream Team URL    79
Match ID          76
dtype: int64
-----------------------------------------


In [5]:
dfDTLinks = dfDTLinks.dropna(subset=["Dream Team URL"])
print("-----------------------------------------")
print("DataFrame Info")
print(dfDTLinks.info())
print("-----------------------------------------\nDataFrame Null Values")
print(dfDTLinks.isnull().sum())
print("-----------------------------------------")

-----------------------------------------
DataFrame Info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 542 entries, 0 to 544
Data columns (total 3 columns):
Event             542 non-null object
Dream Team URL    542 non-null object
Match ID          542 non-null float64
dtypes: float64(1), object(2)
memory usage: 16.9+ KB
None
-----------------------------------------
DataFrame Null Values
Event             0
Dream Team URL    0
Match ID          0
dtype: int64
-----------------------------------------


The Match ID column is converted from a float to an int to remove the decimal.

In [6]:
dfDTLinks["Match ID"] = dfDTLinks["Match ID"].astype(int)
dfDTLinks.head(5)

,Event,Dream Team URL,Match ID
0,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,82145
1,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5c89...,82143
2,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5cab...,82123
3,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5c81...,82121
4,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5c88...,82120


An example of the DreamTeam page is printed.

In [7]:
print(dfDTLinks.iloc[0]["Dream Team URL"])

https://dreamteam.gg/csgo/analytics/match/5ca233b060aed2024a668693/scoreboard


DreamTeam loads statistics onto the page from an API after loading the intial webpage. An example of this is "https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/MATCHCODE/entryfrags", where MATCHCODE is the alphanumeric identification code that comes before scoreboard in the original URL. So the MATCHCODE is needed from each URL in the dfDTLinks DataFrame.

Using the split function, each URL is split by "/". The list is converted to a DataFrame, and then the 7th column of this DataFrame is appended to the dfDTLinks DataFrame under the new column: "MATCHCODE".

In [8]:
dfSplitURL = dfDTLinks["Dream Team URL"].str.split("/")
dfSplitURL = dfSplitURL.apply(pd.Series)
dfSplitURL.head(5)

,0,1,2,3,4,5,6,7
0,https:,,dreamteam.gg,csgo,analytics,match,5ca233b060aed2024a668693,scoreboard
1,https:,,dreamteam.gg,csgo,analytics,match,5c89261a56ab9e00774eb492,scoreboard
2,https:,,dreamteam.gg,csgo,analytics,match,5cab5fdc8da20e066d34d792,scoreboard
3,https:,,dreamteam.gg,csgo,analytics,match,5c812acca09b3b01057e9117,scoreboard
4,https:,,dreamteam.gg,csgo,analytics,match,5c887e2f50247d0762798b32,scoreboard


In [9]:
dfDTLinks["MATCHCODE"] = dfSplitURL[6]
dfDTLinks.head(5)

,Event,Dream Team URL,Match ID,MATCHCODE
0,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,82145,5ca233b060aed2024a668693
1,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5c89...,82143,5c89261a56ab9e00774eb492
2,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5cab...,82123,5cab5fdc8da20e066d34d792
3,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5c81...,82121,5c812acca09b3b01057e9117
4,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5c88...,82120,5c887e2f50247d0762798b32


The MATCHCODE column is then converted in a list for use in scraping later.

In [10]:
matchcodeList = dfDTLinks["MATCHCODE"]
matchcodeList

0      5ca233b060aed2024a668693
1      5c89261a56ab9e00774eb492
2      5cab5fdc8da20e066d34d792
3      5c812acca09b3b01057e9117
4      5c887e2f50247d0762798b32
5      5ca22dcfbcf70a024001c0c3
6      5ca22e408da20e026a4f0fc3
7      5ca23570d35cfb023c7b2d92
8      5c7a93914bd41105690c4472
9      5cab600bdbb10b066f55dfe2
10     5ca257b260aed202884dcb62
11     5c78bd9113771d03f809d692
12     5c7a93c1ac1fca051d27f453
13     5ca2589cbcf70a02726f6a24
14     5c9813090314b40aec21a2d4
15     5c75901372e92601bc59a4d3
16     5ca259548da20e029221d282
17     5ca2597260aed20279507b62
18     5cab60c36bd0f9062e2d4245
19     5cab60e36bd0f9065050ac53
20     5ca25adc8da20e029221d283
21     5ca25bb8bcf70a02726f6a26
22     5ca25cfbd43602024526e6b2
23     5ca25d1560aed202884dcb64
24     5ca25d26d43602024526e6b3
25     5ca25dcd60aed200434a88a5
26     5ca25ddbc0f32a02571dd612
27     5ca25df260aed20279507b63
28     5ca25e54bcf70a02726f6a27
29     5ca25e8160aed20279507b64
                 ...           
515    5

DreamTeam's API stores data in JSON form, so instead of using BeautifulSoup, the function json.loads() is used instead. An example is retrieved and printed in JSON format using PrettyPrint. The JSON holds an item for each round which tells which team one the round and various other factors.

In [11]:
# Form URL
testURL = get_url(matchcodeList[0])

# Get JSON data
data = get_json(testURL)

# Pretty Printer to show example of JSON output
pprint(data)

{'items': [{'number': 1,
            'reason': 'terrorist_win',
            'teams': [{'name': 'Astralis', 'side': 'terrorist'},
                      {'name': 'ENCE', 'side': 'ct'}],
            'winner_side': 'terrorist',
            'winner_team': 'Astralis'},
           {'number': 2,
            'reason': 'terrorist_win',
            'teams': [{'name': 'Astralis', 'side': 'terrorist'},
                      {'name': 'ENCE', 'side': 'ct'}],
            'winner_side': 'terrorist',
            'winner_team': 'Astralis'},
           {'number': 3,
            'reason': None,
            'teams': [{'name': 'Astralis', 'side': 'terrorist'},
                      {'name': 'ENCE', 'side': 'ct'}],
            'winner_side': None,
            'winner_team': None},
           {'number': 4,
            'reason': 'terrorist_win',
            'teams': [{'name': 'Astralis', 'side': 'terrorist'},
                      {'name': 'ENCE', 'side': 'ct'}],
            'winner_side': 'terrorist',
        

The full round winner dataset is then scraped and the Match ID and Event for each corresponding match is taken from the dfDTLinks DataFrame and appended.

In [12]:
# Round winner dataframe
dfRoundWinner = pd.DataFrame(columns=["Event", "Match ID", "Round", "Winner", "Winner Side", "Match Winner"])

# For each row in DTLinks and using the scrape function, scrape the JSON data for each match and event
for _, row in dfDTLinks.iterrows():
    dfRoundWinner = dfRoundWinner.append(scrape_round_winner(row["Match ID"], row["Event"], row["MATCHCODE"]))
    
dfRoundWinner

2019-05-10 19:46:32.539 | INFO     | __main__:scrape_round_winner:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5ca233b060aed2024a668693/rounds


2019-05-10 19:46:34.039 | INFO     | __main__:scrape_round_winner:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5c89261a56ab9e00774eb492/rounds


2019-05-10 19:46:35.544 | INFO     | __main__:scrape_round_winner:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5cab5fdc8da20e066d34d792/rounds


2019-05-10 19:46:37.051 | INFO     | __main__:scrape_round_winner:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5c812acca09b3b01057e9117/rounds


2019-05-10 19:46:38.579 | INFO     | __main__:scrape_round_winner:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5c887e2f50247d0762798b32/rounds


...

2019-05-10 20:00:07.532 | INFO     | __main__:scrape_round_winner:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5caa398cdbb10b05f37119b4/rounds


2019-05-10 20:00:08.990 | INFO     | __main__:scrape_round_winner:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5caa39bbd43602057e2452d7/rounds


2019-05-10 20:00:10.468 | INFO     | __main__:scrape_round_winner:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5caa39bf3cb88705cc46dd23/rounds


2019-05-10 20:00:11.938 | INFO     | __main__:scrape_round_winner:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5caa39c3d436020565490336/rounds


,Event,Match ID,Round,Winner,Winner Side,Match Winner
0,IEM Katowice 2019,82145,1,Astralis,terrorist,Astralis
1,IEM Katowice 2019,82145,2,Astralis,terrorist,Astralis
2,IEM Katowice 2019,82145,3,None,None,Astralis
3,IEM Katowice 2019,82145,4,Astralis,terrorist,Astralis
4,IEM Katowice 2019,82145,5,Astralis,terrorist,Astralis
5,IEM Katowice 2019,82145,6,Astralis,terrorist,Astralis
6,IEM Katowice 2019,82145,7,None,None,Astralis
7,IEM Katowice 2019,82145,8,Astralis,terrorist,Astralis
8,IEM Katowice 2019,82145,9,None,None,Astralis
9,IEM Katowice 2019,82145,10,Astralis,terrorist,Astralis


## Cleaning the Scraped Data
The data collected is then checked to see if there are any discrepancies. Firstly, the amount of null values for each column are printed.

In [13]:
dfRoundWinner

,Event,Match ID,Round,Winner,Winner Side,Match Winner
0,IEM Katowice 2019,82145,1,Astralis,terrorist,Astralis
1,IEM Katowice 2019,82145,2,Astralis,terrorist,Astralis
2,IEM Katowice 2019,82145,3,None,None,Astralis
3,IEM Katowice 2019,82145,4,Astralis,terrorist,Astralis
4,IEM Katowice 2019,82145,5,Astralis,terrorist,Astralis
5,IEM Katowice 2019,82145,6,Astralis,terrorist,Astralis
6,IEM Katowice 2019,82145,7,None,None,Astralis
7,IEM Katowice 2019,82145,8,Astralis,terrorist,Astralis
8,IEM Katowice 2019,82145,9,None,None,Astralis
9,IEM Katowice 2019,82145,10,Astralis,terrorist,Astralis


In [14]:
print("-----------------------------------------\nDataFrame Null Values")
print(dfRoundWinner.isnull().sum())
print("-----------------------------------------")

-----------------------------------------
DataFrame Null Values
Event             0
Match ID          0
Round             0
Winner          557
Winner Side     554
Match Winner      0
dtype: int64
-----------------------------------------


As shown above, there are over 550 missing values for the Winner and Winner Side columns. However, they do not have the same amount of missing values. After investigating the data collected, it is show to be an error in the JSON data retrieved from DreamTeam. For example, in some cases there were more round items than actually occured in the match and these items held mostly null values.

To solve this, a new DataFrame is created which is the scraped data grouped by Match ID and the values are counted.

In [15]:
dfRoundWinnerCheck = (dfRoundWinner.groupby("Match ID").count())
dfRoundWinner.groupby("Match ID").count()

,Event,Round,Winner,Winner Side,Match Winner
Match ID,,,,,
17523,23,23,23,23,23
17524,17,17,17,17,17
17525,20,20,20,20,20
17526,18,18,18,18,18
17527,22,22,22,22,22
17528,21,21,21,21,21
17529,23,23,23,23,23
17530,23,23,23,23,23
17531,24,24,23,23,24


A mask is then used to only keep the matches where the Round column value is equal to the Winner column value.

In [16]:
dfRoundWinnerCheck = dfRoundWinnerCheck[dfRoundWinnerCheck["Round"]==dfRoundWinnerCheck["Winner"]]
dfRoundWinnerCheck

,Event,Round,Winner,Winner Side,Match Winner
Match ID,,,,,
17523,23,23,23,23,23
17524,17,17,17,17,17
17525,20,20,20,20,20
17526,18,18,18,18,18
17527,22,22,22,22,22
17528,21,21,21,21,21
17529,23,23,23,23,23
17530,23,23,23,23,23
17532,25,25,25,25,25


The Match IDs of the remaining matches are then stored in a new DataFrame.

In [17]:
dfCleanIDs = pd.DataFrame(dfRoundWinnerCheck.index.values, columns=["Match ID"])
dfCleanIDs

,Match ID
0,17523
1,17524
2,17525
3,17526
4,17527
5,17528
6,17529
7,17530
8,17532
9,17533


The Match ID column is converted to a numeric, this allows dfRoundWinner and dfCleanIDs to be merged on this column as they are now the same object type.

This merge only leaves the valid matches remaining in dfRoundWinnerClean.

In [18]:
dfRoundWinner["Match ID"] = pd.to_numeric(dfRoundWinner["Match ID"])

dfRoundWinnerClean = pd.merge(dfRoundWinner, dfCleanIDs, on=["Match ID"], how="inner")
dfRoundWinnerClean

,Event,Match ID,Round,Winner,Winner Side,Match Winner
0,IEM Katowice 2019,82050,1,FaZe Clan,ct,Natus Vincere
1,IEM Katowice 2019,82050,2,FaZe Clan,ct,Natus Vincere
2,IEM Katowice 2019,82050,3,Natus Vincere,terrorist,Natus Vincere
3,IEM Katowice 2019,82050,4,Natus Vincere,terrorist,Natus Vincere
4,IEM Katowice 2019,82050,5,Natus Vincere,terrorist,Natus Vincere
5,IEM Katowice 2019,82050,6,Natus Vincere,terrorist,Natus Vincere
6,IEM Katowice 2019,82050,7,FaZe Clan,ct,Natus Vincere
7,IEM Katowice 2019,82050,8,FaZe Clan,ct,Natus Vincere
8,IEM Katowice 2019,82050,9,Natus Vincere,terrorist,Natus Vincere
9,IEM Katowice 2019,82050,10,Natus Vincere,terrorist,Natus Vincere


The same merge is carrid out on dfDTLinks, so that in the next scrape only valid matches are collected.

In [19]:
dfDTLinksClean = pd.merge(dfDTLinks, dfCleanIDs, on=["Match ID"], how="inner")
dfDTLinksClean

,Event,Dream Team URL,Match ID,MATCHCODE
0,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,82050,5ca2589cbcf70a02726f6a24
1,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,81796,5ca260136bd0f9026b37cb74
2,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,81795,5ca260cd27b8d8026d731022
3,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,81790,5ca260d3d35cfb023c7b2d94
4,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,81761,5ca262c227b8d8026d731023
5,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5cac...,74129,5cac8ef6d43602068832b752
6,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74128,5ca273ecd43602024526e6b5
7,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74061,5ca273f3d35cfb026f6f35e2
8,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74060,5ca273fb60aed202884dcb67
9,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74059,5ca274bbbcf70a0040573a99


The cleaned DreamTeam DataFrame and round winner DataFrame are saved locally in CSV files.

In [20]:
dfDTLinksClean.to_csv("DTLINKS.csv", header=True, index=True)
dfRoundWinnerClean.to_csv("ROUNDWINNER.csv", header=True, index=True)